In [33]:
from glob import glob
from astropy.table import Table
import astropy.units as u

Download is just done by hand. Can look at POST request, see the fields in the that request and automate the process of querying, but for now, I'm just saving the time and it's quicker to download by hand.

In [34]:
xrayf = glob('/Users/guenther/Downloads/xray*.dat')
xrayf.sort()

In [35]:
def read_CXRO_file(filename):
    tab = Table.read(filename, format='ascii.no_header', data_start=2, names=['energy', 'reflectivity'])
    with open(filename) as f:
        line = f.readline()
    # Could be a little more clever with re and check that the other paraemters in the header (e.g. the material!)
    # are actually the same for all files, but this is fasted for now.
    angle = line.split(',')[-1].strip().replace('deg','')
    return tab, angle

In [41]:
tab = Table.read(xrayf[0], format='ascii.no_header', data_start=2, names=['energy', 'reflectivity'])
combined = Table(tab[['energy']])

for f in xrayf:
    tab, angle = read_CXRO_file(f)
    combined[angle] = tab['reflectivity']

combined['energy'].unit = u.eV
combined.meta['CREATOR'] = 'Moritz Guenther/MIT'
combined.meta['HENKE'] = 'Ir Rho=22.42, Sig=0.nm, P=0.'
combined.meta['NOTES'] = 'Retrieved from CXRO online query form henke'

In [42]:
combined.write('/Users/guenther/projects/lynx/marxs-lynx/marxslynx/data/ir_refelctivity.dat', format='ascii.ecsv')